In [1]:
from collections import Counter
import json
import numpy as np
import random

In [7]:
# --- Load from dataset ---
data = np.load("gestures.npz", allow_pickle=True)

print("keys:", data.files)
print("static size:", len(data["static"]))
print("dynamic size:", len(data["dynamic"]))
print("sample static entry:", data["static"][0])
print("sample dynamic sequence length:", len(data["dynamic"][0]))

keys: ['static', 'dynamic']
static size: 8354
dynamic size: 199
sample static entry: {'landmarks': [0.0, 0.0, 0.0, -0.11945974147742817, -0.018405142596841624, -0.056197675805242854, -0.22372967639044486, -0.1025255345613317, -0.09017560876443917, -0.30059217464420934, -0.1845819450343788, -0.11956145690886948, -0.36803876801263063, -0.22532925905173726, -0.15155465234130466, -0.15147020063991118, -0.40687224857403587, -0.07595126844219104, -0.2033949488684303, -0.587700612504207, -0.11891824446545243, -0.23282240664495335, -0.7069634306706298, -0.15120686229659427, -0.2533035090368271, -0.8152510383362356, -0.17682047312897683, -0.07779620701853718, -0.4565550776112955, -0.08296690096282645, -0.10267813814905721, -0.6885631991782598, -0.12276982306589869, -0.11682547295210674, -0.8422990959420549, -0.1546485583211, -0.1260109152763951, -0.9754907047625436, -0.1803860697312326, -0.007545612562936476, -0.4463157242457175, -0.09631797054834883, -0.0017150535078916162, -0.6714139817127385

In [ ]:
# --- Parameters ---
SEQ_LEN = 10
num_features = 63
label_map = {"Idle": 0, "Aim": 1, "Fire": 2}
num_classes = len(label_map)

In [8]:
# --- Split by class ---
# Load the main gestures file
data_file = "gestures.npz"
data = np.load(data_file, allow_pickle=True)

static = data["static"].tolist()
dynamic = data["dynamic"].tolist()

# --- Helper to save raw data ---
def save_class_rawdata(filename, rawdata):
    arr = np.array(rawdata, dtype=object)
    np.savez_compressed(filename, rawdata=arr)
    print(f"Saved {len(rawdata)} items to {filename}")

# --- Split static data by label ---
static_classes = {}
for item in static:
    label = item["label"]
    static_classes.setdefault(label, []).append(item)

for label, rawdata in static_classes.items():
    save_class_rawdata(f"static_{label}_rawdata.npz", rawdata)

# --- Split dynamic data by label ---
dynamic_classes = {}
for item in dynamic:
    label = item["label"]
    dynamic_classes.setdefault(label, []).append(item)

for label, rawdata in dynamic_classes.items():
    save_class_rawdata(f"dynamic_{label}_rawdata.npz", rawdata)

Saved 2675 items to static_Idle_rawdata.npz
Saved 5679 items to static_Aim_rawdata.npz
Saved 200 items to dynamic_Fire_rawdata.npz


In [9]:
SEQUENCE_LENGTH = 10  # frames per sequence

# List of static rawdata files
static_files = [
    "static_Idle_rawdata.npz",
    "static_Aim_rawdata.npz"
]

for file in static_files:
    data = np.load(file, allow_pickle=True)
    rawdata = data["rawdata"].tolist()

    sequences = []
    labels = []

    # Sliding window over rawdata
    for i in range(len(rawdata) - SEQUENCE_LENGTH + 1):
        chunk = rawdata[i:i + SEQUENCE_LENGTH]
        sequences.append([item["landmarks"] for item in chunk])
        labels.append(chunk[0]["label"])  # label is the same for all frames in chunk

    # Save the sequences
    output_file = file.replace("_rawdata", "_sequences")
    np.savez_compressed(output_file, sequences=np.array(sequences, dtype=object), labels=np.array(labels, dtype=object))
    print(f"Saved {len(sequences)} sequences to {output_file}")


Saved 2666 sequences to static_Idle_sequences.npz
Saved 5670 sequences to static_Aim_sequences.npz


In [10]:
SEQUENCE_LENGTH = 10  # frames per sequence

# Dynamic rawdata file(s)
dynamic_files = ["dynamic_Fire_rawdata.npz"]

for file in dynamic_files:
    data = np.load(file, allow_pickle=True)
    rawdata = data["rawdata"].tolist()

    sequences = []
    labels = []

    for seq in rawdata:
        frames = seq["landmarks"]
        label = seq["label"]
        n_frames = len(frames)

        if n_frames < SEQUENCE_LENGTH:
            # Pad with zeros
            pad_length = SEQUENCE_LENGTH - n_frames
            # Assume each frame is 63 landmarks (like your normalized landmarks)
            zero_frame = [0.0] * len(frames[0])
            padded_frames = frames + [zero_frame] * pad_length
            sequences.append(padded_frames)
            labels.append(label)

        elif n_frames == SEQUENCE_LENGTH:
            # Keep as-is
            sequences.append(frames)
            labels.append(label)

        else:
            # Create overlapping sequences
            for i in range(n_frames - SEQUENCE_LENGTH + 1):
                chunk = frames[i:i + SEQUENCE_LENGTH]
                sequences.append(chunk)
                labels.append(label)

    # Save the sequences
    output_file = file.replace("_rawdata", "_sequences")
    np.savez_compressed(output_file, sequences=np.array(sequences, dtype=object), labels=np.array(labels, dtype=object))
    print(f"Saved {len(sequences)} sequences to {output_file}")


Saved 1159 sequences to dynamic_Fire_sequences.npz


In [12]:
SEQ_LEN = 10

# Load NPZ file
file = "dynamic_Fire_sequences.npz"
data_npz = np.load(file, allow_pickle=True)
data = data_npz["sequences"].tolist()  # list of sequences
labels = data_npz["labels"].tolist()    # corresponding labels

# --- Filter sequences to exact SEQ_LEN ---
filtered_sequences = []
filtered_labels = []
removed_counts = Counter()

for seq, label in zip(data, labels):
    if len(seq) == SEQ_LEN:
        filtered_sequences.append(seq)
        filtered_labels.append(label)
    else:
        removed_counts[label] += 1

print("Sequences removed due to length != 10:", removed_counts)
print("Sequences kept per class:", Counter(filtered_labels))

# --- Save filtered sequences back to NPZ ---
np.savez_compressed(
    file.replace(".npz", "_filtered.npz"),
    sequences=np.array(filtered_sequences, dtype=object),
    labels=np.array(filtered_labels, dtype=object)
)

Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 1})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 2})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 3})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 4})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 5})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 6})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 7})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 8})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 9})
Sequences removed due to length != 10: Counter()
Sequences kept per class: Counter({'Fire': 10})
Sequences removed due to length != 10: 

In [ ]:
def augment_fire_sequences(sequences, target_count, jitter=2, noise_std=0.01, frame_dropout_prob=0.1):
    """
    Augment short dynamic gesture sequences to increase dataset size.

    Args:
        sequences: List of dicts, each {"landmarks": [[...]], "label": "Fire"}.
        target_count: Desired total number of sequences after augmentation.
        jitter: Max frames to shift start/end.
        noise_std: Standard deviation of Gaussian noise added to landmarks.
        frame_dropout_prob: Probability to randomly drop a frame.

    Returns:
        Augmented list of sequences.
    """
    augmented = sequences.copy()
    while len(augmented) < target_count:
        seq = random.choice(sequences)
        landmarks = seq["landmarks"]
        seq_len = len(landmarks)

        # --- Temporal jittering ---
        start_shift = random.randint(0, min(jitter, seq_len-1))
        end_shift = random.randint(0, min(jitter, seq_len-1))
        new_seq = landmarks[start_shift: seq_len - end_shift]

        # --- Frame dropout / repetition ---
        frames = []
        for frame in new_seq:
            if random.random() < frame_dropout_prob and len(new_seq) > 1:
                continue  # drop this frame
            frames.append(frame)
        if len(frames) == 0:
            frames = new_seq.copy()

        # --- Add small Gaussian noise ---
        noisy_frames = []
        for frame in frames:
            frame_array = np.array(frame)
            frame_array += np.random.normal(0, noise_std, frame_array.shape)
            noisy_frames.append(frame_array.tolist())

        augmented.append({
            "landmarks": noisy_frames,
            "label": seq["label"]
        })

    return augmented

In [14]:
from random import shuffle

def sample_shuffle_and_save(class_files, output_file, sample_size_per_class=None, random_seed=None):
    """
    Load sequences from multiple NPZ files (one per class), optionally sample a fixed number per class,
    shuffle them together, and save to a new NPZ file.

    Args:
        class_files (list of str): List of NPZ file paths, one per class.
        output_file (str): Path for the output NPZ file.
        sample_size_per_class (int or None): Max sequences to sample per class. If None, use all sequences.
        random_seed (int or None): Seed for reproducibility.

    Returns:
        None
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    sequences = []
    labels = []

    for file in class_files:
        data_npz = np.load(file, allow_pickle=True)
        class_sequences = data_npz["sequences"].tolist()
        class_labels = data_npz["labels"].tolist()

        # Sample if needed
        if sample_size_per_class is not None and len(class_sequences) > sample_size_per_class:
            indices = np.random.choice(len(class_sequences), size=sample_size_per_class, replace=False)
            class_sequences = [class_sequences[i] for i in indices]
            class_labels = [class_labels[i] for i in indices]

        sequences.extend(class_sequences)
        labels.extend(class_labels)

    # Shuffle together
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)

    # Save to NPZ
    np.savez_compressed(output_file,
                        sequences=np.array(sequences, dtype=object),
                        labels=np.array(labels, dtype=object))
    print(f"Saved {len(sequences)} shuffled sequences to {output_file}")

In [15]:
class_files = ["static_Idle_sequences.npz", "static_Aim_sequences.npz", "dynamic_Fire_sequences.npz"]
output_file = "all_classes_shuffled.npz"

sample_shuffle_and_save(class_files, output_file, sample_size_per_class=1159, random_seed=42)

Saved 3477 shuffled sequences to all_classes_shuffled.npz
